<a href="https://colab.research.google.com/github/stefanogiagu/corso_AI_2022/blob/main/prova8_9/MetodiAI_Fis2022_ProvaPratica_8_9_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prova Pratica - Metodi AI e Machine Learning per la fisica 
### 08.07.2022 - AA 2021/22 - Docente: S. Giagu


> **Regole:**

*   **tempo a disposizione:** 3h

*   compilare con i vostri dati i campi della cella che segue e poi eseguire la cella verificando che i dati printati corrispondano
*   risolvere i quesiti/compiti indicati nella cella *Descrizione del compito* usando questo notebook 
*   una volta completato il compito sottomettere il notebook nel apposito folder sul sito e-learning del corso disponibile al link: <p>
[consegna esonero](https://elearning.uniroma1.it/mod/assign/view.php?id=488449) <p>
NOTA BENE: una volta sottomesso il notebook non sono più possibili ulteriori modifiche

In [ ]:
#@ Dati Personali
import os

Nome = 'Stefano'  #@param {type: "string"}
Cognome = 'Giagu' #@param {type: "string"}
NumeroMatricola = 12345678 #@param {type: "number"}

if NumeroMatricola == 12345678:
  print('\033[1;31m Inserisci il numero di matricola corretto!!!!') 
else:
  !rm -rf data_mlai_8.9.2022.npz
  !wget http://giagu.web.cern.ch/giagu/CERN/data_mlai_8.9.2022.npz
  print('Done')

 Inserisci il numero di matricola corretto!!!!


# Descrizione del compito:

### Dataset:

Il dataset *data_mlai_8.9.2022.npz* è in formato numpy compresso e contiene due array numpy *x* (vettore dati) e *y* (vettore label), di shape: 

```
x shape: (20800, 56, 56)
y shape: (20800,)
```

*x* contiene 20800 immagini di caratteri (lettere) scritte a mano, in scala di grigi, *y* contiene una label numerica che identifica il carattere (numero tra 1 e 26). 

**Compiti:**

0.   leggere in numpy i due array e analizzare le caratteristiche che si ritengono più significative del dataset, plottando 9 immagini esempio del campione
1.   nel notebook è fornito un esempio (classe *Compressore*) di modulo di pytroch che funge da compressore per il campione, una rete convoluzionale che prende l'immagine *56x56* e la comprime in una rappresentazione latente *z* di dimensione *2*. Viene chiesto di scrivere in pytroch un modulo di decompressione che a partire dalla dimensione latente *z* fornisce in input un vettore delle dimensioni dell'immagine originale.
2.   combinare il *Compressore* e *Decompressore* in un modello di AutoEncoder ed allenarlo a ricostruire le immagini del dataset
3.   plottare lo scatter plot dello spazio latente ($z[0]$ vs $z[1]$) per le immagini del dataset ottenute dopo aver applicato il Compressore (allenato nel AutoEncoder), colorando ciascun punto con il valore della label *y*
4.   applicare al campione *x* una analisi PCA con 2 componenti principali, riportando la varianza spiegata da ciascuna componente
5.   plottare lo scatter plot delle due componenti principali, colorando ciascun punto con il valore della label *y*
6.   per confrontare la qualità dei due spazi latenti prodotti dall'AE e dalla PCA, allenare due classificatori lineari di tipo *sklearn.linear_model.LogisticRegression* al task di classificare la label *y* a partire dalla rappresentazione dello spazio latente *z* (per l'AE) e *(pca_0, pca_1)* (per la PCA). Allenando il modello usando 5000 eventi presi dal dataset, e testando l'accuracy media su ulteriori 5000 eventi diversi del dataset

* commentare opportunamente i risultati ottenuti in ogni punto

In [ ]:
# Modulo torch.nn per la compressione delle immagini
# input:  immagine 2D con 1 canale (scala di grigi)
# output: rappresentazione latente z di dimensione 2

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

class Compressore(nn.Module):
    def __init__(self):
        super(Compressore, self).__init__()

        self.c1 = nn.Conv2d(1, 16, 3, stride=2, padding=1)
        self.c2 = nn.Conv2d(16, 32, 3, stride=2, padding=1)
        self.c3 = nn.Conv2d(32, 64, 3, stride=2, padding=1)
        self.c4 = nn.Conv2d(64, 128, 3, stride=2, padding=1)
        self.l1 = nn.Linear(128 * 4* 4, 2)

        self.flatten = nn.Flatten()
        self.activ = nn.LeakyReLU(0.2)

    def forward(self, input):
        x = self.c1(input)
        x = self.activ(x)
        x = self.c2(x)
        x = self.activ(x)
        x = self.c3(x)
        x = self.activ(x)
        x = self.c4(x)
        x = self.activ(x)

        x = self.flatten(x) 

        z = self.l1(x)

        return z

compressore = Compressore()

from torchsummary import summary
if torch.cuda.is_available():
  summary(compressore.cuda(), input_size=(1,56,56))
else:
  summary(compressore, input_size=(1,56,56))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
         LeakyReLU-2           [-1, 16, 28, 28]               0
            Conv2d-3           [-1, 32, 14, 14]           4,640
         LeakyReLU-4           [-1, 32, 14, 14]               0
            Conv2d-5             [-1, 64, 7, 7]          18,496
         LeakyReLU-6             [-1, 64, 7, 7]               0
            Conv2d-7            [-1, 128, 4, 4]          73,856
         LeakyReLU-8            [-1, 128, 4, 4]               0
           Flatten-9                 [-1, 2048]               0
           Linear-10                    [-1, 2]           4,098
Total params: 101,250
Trainable params: 101,250
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.38
Params size (MB): 0.39
Estimated T

In [ ]:
# codice ...